## Deliverable 2. Create a Customer Travel Destinations Map.

In [26]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key_new

# Configure gmaps API key
gmaps.configure(api_key=g_key_new)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Iqaluit,CA,63.7506,-68.5145,46.13,75,90,11.01,overcast clouds
1,1,Albany,US,42.6001,-73.9662,72.75,84,53,6.67,broken clouds
2,2,Thinadhoo,MV,0.5333,72.9333,81.37,75,100,6.96,overcast clouds
3,3,Katete,UG,-0.7818,29.7544,69.48,55,100,1.05,overcast clouds
4,4,Mataura,NZ,-46.1927,168.8643,46.47,97,100,2.39,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Thinadhoo,MV,0.5333,72.9333,81.37,75,100,6.96,overcast clouds
10,10,Vung Tau,VN,10.3460,107.0843,80.56,82,100,14.83,overcast clouds
16,16,Atuona,PF,-9.8000,-139.0333,77.59,68,74,17.94,broken clouds
17,17,Cabo Rojo,PR,18.0866,-67.1457,83.91,87,47,10.80,scattered clouds
19,19,Wattegama,LK,6.7989,81.4808,77.67,69,100,6.22,overcast clouds
24,24,Alofi,NU,-19.0595,-169.9187,78.69,83,75,16.11,broken clouds
26,26,Shingu,JP,33.7333,135.9833,83.39,77,100,8.68,overcast clouds
28,28,San Pedro,PH,14.3500,121.0167,79.16,94,100,16.28,heavy intensity rain
30,30,Nancha,CN,47.1333,129.2667,80.26,73,27,1.36,scattered clouds
34,34,Pedro Carbo,EC,-1.8333,-80.2333,85.21,58,18,9.86,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                259
City                   259
Country                259
Lat                    259
Lng                    259
Max Temp               259
Humidity               259
Cloudiness             259
Wind Speed             259
Current Description    259
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna(axis=0, how='any')
preferred_cities_df.count()

City_ID                259
City                   259
Country                259
Lat                    259
Lng                    259
Max Temp               259
Humidity               259
Cloudiness             259
Wind Speed             259
Current Description    259
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Thinadhoo,MV,81.37,overcast clouds,0.5333,72.9333,
10,Vung Tau,VN,80.56,overcast clouds,10.3460,107.0843,
16,Atuona,PF,77.59,broken clouds,-9.8000,-139.0333,
17,Cabo Rojo,PR,83.91,scattered clouds,18.0866,-67.1457,
19,Wattegama,LK,77.67,overcast clouds,6.7989,81.4808,
24,Alofi,NU,78.69,broken clouds,-19.0595,-169.9187,
26,Shingu,JP,83.39,overcast clouds,33.7333,135.9833,
28,San Pedro,PH,79.16,heavy intensity rain,14.3500,121.0167,
30,Nancha,CN,80.26,scattered clouds,47.1333,129.2667,
34,Pedro Carbo,EC,85.21,few clouds,-1.8333,-80.2333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key_new}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [29]:
# 7. Drop the rows where there is no Hotel Name
# Replace empty hotels to Nan so Pandas can perform dropna 
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace("",np.nan)
hotel_df_clean = hotel_df.dropna(axis=0, how='any')
print(len(hotel_df_clean))
hotel_df_clean.head(10)

216


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Vung Tau,VN,80.56,overcast clouds,10.3460,107.0843,Green hotel
16,Atuona,PF,77.59,broken clouds,-9.8000,-139.0333,Villa Enata
17,Cabo Rojo,PR,83.91,scattered clouds,18.0866,-67.1457,Hotel Perichi's
24,Alofi,NU,78.69,broken clouds,-19.0595,-169.9187,Taloa Heights
26,Shingu,JP,83.39,overcast clouds,33.7333,135.9833,Shingu UI Hotel
28,San Pedro,PH,79.16,heavy intensity rain,14.3500,121.0167,Villa Veronica Resort Private Pool
35,Ushtobe,KZ,75.81,overcast clouds,45.2514,77.9838,"Gostinitsa ""Diyar"""
38,Progreso,MX,84.79,few clouds,21.2833,-89.6667,Playa Linda Hotel
41,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,Keleti Beach Resort
48,Bubaque,GW,81.01,overcast clouds,11.2833,-15.8333,"Casa Dora, Bubaque"


In [46]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df_clean.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame
locations = hotel_df_clean[["Lat", "Lng"]]

In [53]:
# 11a. Add a marker layer for each city to the map
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))